In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install accelerate>=0.20.1
!pip install accelerate -U
!pip install transformers[torch]
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load the entire CSV file using pandas
dataset_file = '/content/drive/MyDrive/Colab Notebooks/data/preprocessed_chatg.csv'
data = pd.read_csv(dataset_file)
data=data[:19998]
# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

print()

# Convert the split data to datasets library format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-base')
max_source = 128
max_target = 128

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_source)
    tokenized_label = tokenizer(batch['paraphrases'], padding='max_length', truncation=True, max_length=max_target)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=512)
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Map:   0%|          | 0/17998 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:

train_dataset.save_to_disk('your/train/save/dir')
val_dataset.save_to_disk('your/val/save/dir')

Saving the dataset (0/1 shards):   0%|          | 0/17998 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-base')
import pandas as pd

dataset_file = '/content/drive/MyDrive/Colab Notebooks/data/preprocessed_chatg.csv'
df = pd.read_csv(dataset_file)
df=df[:19998]
source_text = df['text']
target_text = df['paraphrases']

tokenized_source_text = tokenizer(list(source_text), truncation=False, padding=False)
tokenized_target_text = tokenizer(list(target_text), truncation=False, padding=False)

max_source = 0
for item in tokenized_source_text['input_ids']:
    if len(item) > max_source:
        max_source = len(item)

max_target = 0
for item in tokenized_target_text['input_ids']:
    if len(item) > max_target:
        max_target = len(item)

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained('t5-base')

output_dir = 'your/output/dir'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_name', # Wandb run name
    logging_steps=1000, # How often to log loss to wandb
    eval_steps=1000, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
trainer.save_model(output_dir + '/model')

Step,Training Loss,Validation Loss
1000,0.259300,0.189632
2000,0.216500,0.182858
3000,0.165400,0.187319
4000,0.155400,0.185252
5000,0.127700,0.193071
6000,0.108900,0.191553
7000,0.104000,0.204790
8000,0.073600,0.203612
9000,0.080500,0.201316
10000,0.050200,0.221911


In [ ]:
trainer.save_model('/content/drive/MyDrive/model/model2')

In [ ]:
from transformers import T5ForConditionalGeneration

# Define the directory where you saved the trained model
model_directory = '/content/drive/MyDrive/model/model2'

# Load the trained model
loaded_model = T5ForConditionalGeneration.from_pretrained(model_directory)

# Now, you can use the loaded model for prediction
input_text = "What goes through a guy's mind when he touches his girlfriend?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')  # Make sure to use the same tokenizer used during training

# Generate predictions
output = loaded_model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


What is the thought process when a guy touches his girlfriend?
